In [8]:
import pandas as pd
from google.cloud import bigquery
import re
import numpy as np
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2 import service_account


SERVICE_ACCOUNT_FILE = 'keysMCCSheet.json'
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]
# Pour MCC
MYCLIENTID = "https://docs.google.com/spreadsheets/d/1NFYLNdJMqc7vHKKDWfelQD46EeWLcZwRS4D43Xfh4ug/edit#gid=0" 
MYCLIENTSECRET = "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" 

creds = None
creds = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes = SCOPES)
def googleSheetImport(url_gsheet):
    service = build('sheets', 'v4', credentials=creds)

    # Call Sheets API

    x = url_gsheet
    id_gsheet = re.search('d\/([a-zA-Z0-9_-]+)',x).group(1)
    gid_gsheet = re.search('gid=([a-zA-Z0-9_-]+)',x).group(1)
    # Call Sheets API
    service = build('sheets', 'v4', credentials=creds)
    sheet_metadata = service.spreadsheets().get(spreadsheetId=id_gsheet).execute()
    sheets = sheet_metadata.get('sheets', '')
    sheet = service.spreadsheets()
    #boucle sur les pages du google sheet pour récupérer l'id de page
    for i in sheets : 
        title = i['properties']['title']
        sheet_id = i['properties']['sheetId']
        #si l'id de la feuille est celle demandée
        if gid_gsheet == str(sheet_id) :
            #Obtenir les valeurs du Google Sheet
            result = sheet.values().get(spreadsheetId = id_gsheet, range = title).execute()
            values = result.get('values')  
            return pd.DataFrame(columns = values[0], data = values[1:len(values)])
        #La feuille n'est pas celle demandée
        else :
            1

    
    

In [21]:
def googleSheetExport(produit, url) :
    #Passer du dataframe au format adequat pour l'export sur googlesheet

    produit = produit.fillna(0)
    Data = []
    Weight = produit.values.tolist()
    ColumnName = list(produit.keys())
    Data.append(ColumnName)
    for i in Weight :
        Data.append(i)

    #Format de l'url
    x = url

    id_gsheet = re.search('d\/([a-zA-Z0-9_-]+)',x).group(1)
    gid_gsheet = re.search('gid=([a-zA-Z0-9_-]+)',x).group(1)
    # Call Sheets API
    service = build('sheets', 'v4', credentials=creds)
    sheet = service.spreadsheets()
    sheet_metadata = service.spreadsheets().get(spreadsheetId=id_gsheet).execute()
    sheets = sheet_metadata.get('sheets', '')
    #boucle sur les pages du google sheet pour récupérer l'id de page

    for i in sheets : 

        title = i['properties']['title']
        sheet_id = i['properties']['sheetId'] 
        if gid_gsheet == str(sheet_id) :
            NAME_PAGE = title
            print('title')
            #Timeout configuré pour éviter les bugs
            import socket
            socket.setdefaulttimeout(5000)

            #Supprimer les données présentes avant l'export
            clear = sheet.values().clear(spreadsheetId = id_gsheet,
                                            range = NAME_PAGE).execute()   



            #Exporter les données dans le Google Sheet
            request = sheet.values().update(spreadsheetId = id_gsheet, 
                                            range = NAME_PAGE, 
                                            valueInputOption = "USER_ENTERED", 
                                            body = {"values" : Data}).execute()




In [9]:
df = googleSheetImport('https://docs.google.com/spreadsheets/d/1eKjKnVVmepvl316ArNZkRzW0917V1DHlRBeYfss-tvI/edit?gid=0#gid=0')

In [16]:
valeurs_a_supprimer = ['-']

df = df[~df['VILLE'].isin(valeurs_a_supprimer)]

{'AGEN',
 'AIX-EN-PROVENCE',
 'AIX-LES-BAINS',
 'ALBARET-SAINTE-MARIE',
 'ALBI',
 'ALÈS',
 'AMILLY',
 'AMNÉVILLE',
 'ANGLET',
 'ANNECY',
 'ANNEMASSE',
 'APPOIGNY',
 'ARCUEIL',
 'ARGENTEUIL',
 'ARLES',
 'ARMBOUTS-CAPPEL',
 'ARNAGE',
 'ARPAJON',
 'ARTIGUES-PRÈS-BORDEAUX',
 'ARVEYRES',
 'ATHIS-MONS',
 'AUBAGNE',
 'AUBIÈRE',
 'AUCH',
 'AUGNY',
 'AURILLAC',
 'AVERMES',
 'AVIGNON',
 'AYTRÉ',
 'BAGNOLET',
 'BALARUC-LE-VIEUX',
 'BALLAN-MIRÉ',
 'BARENTIN',
 'BAYEUX',
 'BAYONNE',
 'BEAUCOUZÉ',
 'BEAUNE',
 'BEAUVAIS',
 'BELFORT',
 'BELLERIVE-SUR-ALLIER',
 'BERGERAC',
 'BESANÇON',
 'BEZONS',
 'BIARRITZ',
 'BLAGNAC',
 'BLOIS',
 'BOBIGNY',
 'BOLLÈNE',
 'BONNEUIL-SUR-MARNE',
 'BORDEAUX',
 'BOUAYE',
 'BOUC-BEL-AIR',
 'BOULAZAC',
 'BOULAZAC ISLE MANOIRE',
 'BOURG-EN-BRESSE',
 'BOURG-LÈS-VALENCE',
 'BOURGES',
 'BOURGOIN-JALLIEU',
 'BREST',
 'BRIGNOLES',
 'BRIVE-LA-GAILLARDE',
 'BRON',
 'BUC',
 'BUCHÈRES',
 'BULGNÉVILLE',
 'BUSSY-SAINT-GEORGES',
 'BÈGLES',
 'BÉZIERS',
 'CABRIÈS',
 'CAEN',
 'CAHORS',
 'CA

In [17]:
import sys
import urllib.parse
import requests
import pandas as pd

BASE_URI = "https://weather.lewagon.com"

def search_city(query):
    '''
    Look for a given city. If multiple options are returned, return the first match.
    '''
    url = urllib.parse.urljoin(BASE_URI, "/geo/1.0/direct")
    cities = requests.get(url, params={'q': query, 'limit': 1}).json()

    if not cities:
        print(f"Sorry, OpenWeather does not know about {query}!")
        return None

    return cities[0]

def weather_forecast(lat, lon):
    '''Return a 5-day weather forecast for the city, given its latitude and longitude.'''
    url = urllib.parse.urljoin(BASE_URI, "/data/2.5/forecast")
    forecasts = requests.get(url, params={'lat': lat, 'lon': lon, 'units': 'metric'}).json()['list']

    # Filter forecasts to keep only one per day (e.g., the first entry of each day)
    unique_forecasts = {}
    for forecast in forecasts:
        date = forecast['dt_txt'][:10]
        if date not in unique_forecasts:
            unique_forecasts[date] = forecast

    # Return only J and J+1 forecasts
    forecasts_list = list(unique_forecasts.values())[:2]
    
    # Format the results as dictionaries for easy access later
    j_forecast = {
        'weather': forecasts_list[0]['weather'][0]['main'],
        'temp_max': round(forecasts_list[0]['main']['temp_max'])
    }
    j_plus_1_forecast = {
        'weather': forecasts_list[1]['weather'][0]['main'],
        'temp_max': round(forecasts_list[1]['main']['temp_max'])
    }
    
    return j_forecast, j_plus_1_forecast

def get_weather_for_dataframe(df):
    '''Takes a dataframe with a 'VILLE' column and returns weather forecast for J and J+1'''
    meteo_j = []
    meteo_j_plus_1 = []

    for _, row in df.iterrows():
        city_name = row['VILLE']
        city = search_city(city_name)

        if city:
            j_forecast, j_plus_1_forecast = weather_forecast(city['lat'], city['lon'])
            meteo_j.append(f"{j_forecast['weather']} ({j_forecast['temp_max']}°C)")
            meteo_j_plus_1.append(f"{j_plus_1_forecast['weather']} ({j_plus_1_forecast['temp_max']}°C)")
        else:
            print(f"City {city_name} not found!")
            meteo_j.append(None)
            meteo_j_plus_1.append(None)

    # Add the new columns to the original dataframe
    df['Meteo_J'] = meteo_j
    df['Meteo_J+1'] = meteo_j_plus_1

    return df



In [18]:
df_weather = get_weather_for_dataframe(df)


Sorry, OpenWeather does not know about MONS-EN-BARUL!
City MONS-EN-BARUL not found!
Sorry, OpenWeather does not know about VANDUVRE-LÈS-NANCY!
City VANDUVRE-LÈS-NANCY not found!


C:\Users\lione\AppData\Local\Temp\ipykernel_27252\1145703623.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Meteo_J'] = meteo_j
C:\Users\lione\AppData\Local\Temp\ipykernel_27252\1145703623.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Meteo_J+1'] = meteo_j_plus_1


In [19]:
df_weather

,id,Title,Image link,INVONTORY,VILLE,brand,Volume de recherche,Concurrence,Push,Top Hotel,Top Desti,Meteo_J,Meteo_J+1
0,1,ENZO HOTEL BY KYRIAD DIRECT,https://direct-enzo-loudeac.kyriad.com/fr-fr/,37,LIMOGES,Kyriad,32306,2,YES,Flop,Top,Rain (16°C),Rain (12°C)
1,2,ENZO HOTELS CHARTRES MAINVILLIERS BY KYRIAD DI...,https://direct-enzo-loudeac.kyriad.com/fr-fr/,27,MAINVILLIERS,Kyriad,37737,4,NO,Flop,Top,Rain (16°C),Rain (13°C)
2,3,ENZO HOTELS LOUDEAC BY KYRIAD DIRECT,https://direct-enzo-loudeac.kyriad.com/fr-fr/,114,LOUDÉAC,Kyriad,33755,1,YES,Flop,Top,Clouds (14°C),Clear (9°C)
3,4,ENZO HOTELS METZ AMNÉVILLE BY KYRIAD DIRECT,https://direct-enzo-loudeac.kyriad.com/fr-fr/,14,AMNÉVILLE,Kyriad,42882,5,NO,Flop,Flop,Rain (17°C),Clouds (14°C)
4,5,ENZO HOTELS QUIMPERLE BY KYRIAD DIRECT,https://direct-enzo-loudeac.kyriad.com/fr-fr/,83,QUIMPERLÉ,Kyriad,38011,1,YES,Flop,Flop,Clouds (15°C),Clear (11°C)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
521,522,HÔTEL RESTAURANT CAMPANILE ROUEN NORD BARENTIN,https://direct-enzo-loudeac.kyriad.com/fr-fr/,171,BARENTIN,Campanile,39889,0,YES,Flop,Top,Clouds (15°C),Clouds (14°C)
522,523,HÔTEL RESTAURANT CAMPANILE SAINT-QUENTIN,https://direct-enzo-loudeac.kyriad.com/fr-fr/,1,SAINT-QUENTIN,Campanile,48230,1,YES,Flop,Flop,Rain (16°C),Rain (15°C)
523,524,HÔTEL RESTAURANT CAMPANILE TOULOUSE OUEST PURPAN,https://direct-enzo-loudeac.kyriad.com/fr-fr/,25,TOULOUSE,Campanile,33903,5,NO,Flop,Flop,Rain (19°C),Rain (12°C)
524,525,HÔTEL RESTAURANT CAMPANILE VILLEPINTE PARC DES...,https://direct-enzo-loudeac.kyriad.com/fr-fr/,145,VILLEPINTE,Campanile,15972,1,NO,Flop,Top,Clouds (20°C),Rain (12°C)


In [22]:
googleSheetExport(df_weather, 'https://docs.google.com/spreadsheets/d/1eKjKnVVmepvl316ArNZkRzW0917V1DHlRBeYfss-tvI/edit?gid=867756574#gid=867756574')


title
